In [1]:
import pyvisa
import time
import schedule
import pandas as pd
import serial.tools.list_ports
import os

Port_Arduino = 'COM10'

# Destination folder and file path
folder_path = "C:\\Users\\kary\\Documents\\9_data\\IEAP\\"
file_path = os.path.join(folder_path, "voltage_data.csv")

#Create Resource manager instance
rm = pyvisa.ResourceManager()
#Check the VISA device
rm.list_resources()

('USB0::0x05E6::0x6500::04537333::INSTR', 'ASRL10::INSTR')

In [2]:
# Start the communication by VISA identifying resource name
dmm = rm.open_resource('USB0::0x05E6::0x6500::04537333::INSTR')

#Checking the machine name
print(dmm.query('*IDN?'))

KEITHLEY INSTRUMENTS,MODEL DMM6500,04537333,1.7.11b



In [3]:
dmm.write('*RST')
dmm.write(':DISPlay:VOLTage:DIGits 3')
#dmm.write("SENS:FUNC 'VOLT:DC'")
dmm.write("SENS:FUNC 'VOLT:DC', (@1)")
dmm.write("SENS:FUNC 'RESistance', (@2)")

30

### With Position

In [4]:
#列を5つもつデータフレームに新しい行を追加する
def add_row(df,row):
    new_row = pd.Series({df.columns[0]:row[0],df.columns[1]:row[1],df.columns[2]:row[2]}).to_frame().T   
    # new_row = pd.Series({df.columns[0]:row[0],df.columns[1]:row[1],df.columns[2]:row[2],df.columns[3]:row[3],df.columns[4]:row[4],df.columns[5]:row[5]}).to_frame().T
    return  pd.concat([df,new_row],ignore_index = True)

In [5]:
#データフレーム、マルチメータ、合計測定時間、サンプル周期、印可電圧時間
def df_add_read_data(df,mul,total_read_time,sample_period):
    #測定時間の初期化
    start_time =  time.perf_counter()
    end_time = time.perf_counter()
    while end_time - start_time < total_read_time:
        each_start_time =  time.perf_counter()        
        dmm.write("SENS:FUNC 'VOLT:DC', (@1)")
        data1 = float(mul.query("READ?"))
        dmm.write("SENS:FUNC 'VOLT:DC', (@2)")
        data2 = float(mul.query("READ?"))
        each_end_time = time.perf_counter()
        #サンプル周期になるまで待機
        while  each_end_time - each_start_time <= sample_period:
            each_end_time = time.perf_counter()
            
        df = add_row(df,[each_end_time - start_time,data1,data2])
        end_time = time.perf_counter()
    return df

#### Measure

In [6]:
columns = ["Time[s]","Measured_Voltage[V]","Resistance[Ohm]"]
df_voltage = pd.DataFrame(columns=columns)

position = 0
read_time = 5 # sec
freq = 0.001
df_voltage = df_add_read_data(df_voltage,dmm,read_time,freq)
df_voltage

C:\Users\kary\AppData\Local\Temp\ipykernel_6060\1442998754.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return  pd.concat([df,new_row],ignore_index = True)


,Time[s],Measured_Voltage[V],Resistance[Ohm]
0,0.131540,0.000009,-0.000004
1,0.175456,-0.000015,0.000016
2,0.239455,0.000061,0.000080
3,0.282598,0.000041,0.000044
4,0.348429,0.000075,0.000014
...,...,...,...
88,4.814450,-0.000035,-0.000070
89,4.880883,-0.000051,-0.000034
90,4.926733,0.000009,0.000007
91,4.972738,-0.000019,-0.000005
